In [51]:
import pandas as pd
import numpy as np

from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator

In [52]:
# Parameter setting
np.random.seed(2018)
model_name = 'ResNet50'
img_size = (224, 224)
batch_size = 32
freeze_layer_num = 77

In [53]:
# Data preparation
train_generator = ImageDataGenerator(preprocessing_function = resnet50.preprocess_input,
                                        rotation_range=15,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        shear_range=0.15,
                                        zoom_range=0.15)
valid_test_gen = ImageDataGenerator(preprocessing_function = resnet50.preprocess_input)

train_data_generator = train_generator.flow_from_directory('.\\imgs\\new_train',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          class_mode='categorical')
valid_data_generator = valid_test_gen.flow_from_directory('.\\imgs\\new_valid',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          class_mode='categorical')
test_data_generator = valid_test_gen.flow_from_directory('.\\imgs\\test',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=False,
                                                          class_mode=None)

Found 20271 images belonging to 10 classes.
Found 2153 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [54]:
# Construct model
from keras import backend as K
K.clear_session()

inputs = Input((*img_size, 3))
base_model = ResNet50(input_tensor=inputs, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)

print("total layer count {}".format(len(base_model.layers)))

total layer count 174


In [55]:
# Train the FC layer
base_model.trainabel = False
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_data_generator,
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator,
                    validation_steps=len(valid_data_generator))

Epoch 1/1
127/126 [==============================] - 563s 4s/step - loss: 0.8556 - acc: 0.7219 - val_loss: 0.4081 - val_acc: 0.8709


In [46]:
# fine-tune the model
for i, layer in enumerate(model.layers):
    if i <= freeze_layer_num:
        layer.trainable = False
    else:
        layer.trainable = True
    
model.compile(optimizer=RMSprop(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
127/126 [==============================] - 116s 914ms/step - loss: 0.1985 - acc: 0.9427 - val_loss: 1.3846 - val_acc: 0.8699


In [47]:
# Train model
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
127/126 [==============================] - 853s 7s/step - loss: 0.1188 - acc: 0.9619 - val_loss: 0.5261 - val_acc: 0.9015


In [48]:
# Train model
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
127/126 [==============================] - 527s 4s/step - loss: 0.1091 - acc: 0.9739 - val_loss: 0.3333 - val_acc: 0.9187


In [24]:
if not os.path.exists('.\\models'):
    os.mkdir('.\\models')
model.save('models\\'+model_name+'_L'+ str(freeze_layer_num)+'.h5')

In [50]:
# Train model
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
127/126 [==============================] - 186s 1s/step - loss: 0.0559 - acc: 0.9840 - val_loss: 0.3570 - val_acc: 0.9099


In [40]:
# Train model
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
127/126 [==============================] - 169s 1s/step - loss: 0.0176 - acc: 0.9951 - val_loss: 0.5073 - val_acc: 0.8927


In [49]:
# submition file
df = pd.read_csv("submit_example.csv", index_col='img')
preds = model.predict_generator(test_data_generator, steps=len(test_data_generator), verbose=1)
for i, fname in enumerate(test_data_generator.filenames):
    index = fname[fname.rfind('\\')+1:]
    df.loc[index] = preds[i]

df.to_csv('pred.csv', float_format='%.3f')

2492/2492 [==============================] - 3075s 1s/step
